# Week 2 Day 4 Exercise - Enhanced Airline AI Assistant


This notebook extends the basic airline assistant with a tool to set ticket prices.

### Key Features:
- **Get Ticket Price**: Query current ticket prices for destinations
- **Set Ticket Price**: Update ticket prices for destinations  
- **Database Integration**: Uses SQLite for persistent storage
- **Multiple Tool Support**: Handles both get and set operations
- **Gradio Interface**: User-friendly chat interface


In [1]:
# Import necessary libraries
import os
import json
import sqlite3
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr


In [2]:
# Initialize OpenAI client
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

# System message for the assistant
system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
You can get ticket prices and set ticket prices for different cities.
"""


OpenAI API Key exists and begins sk-proj-


In [3]:
# Database setup
DB = "prices.db"

def setup_database():
    """Initialize the database with the prices table"""
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('CREATE TABLE IF NOT EXISTS prices (city TEXT PRIMARY KEY, price REAL)')
        conn.commit()
    print("✅ Database setup complete!")

# Setup the database
setup_database()


✅ Database setup complete!


In [4]:
# Tool functions
def get_ticket_price(city):
    """Get the price of a ticket to a destination city"""
    print(f"DATABASE TOOL CALLED: Getting price for {city}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE city = ?', (city.lower(),))
        result = cursor.fetchone()
        return f"Ticket price to {city} is ${result[0]}" if result else "No price data available for this city"

def set_ticket_price(city, price):
    """Set the price of a ticket to a destination city"""
    print(f"DATABASE TOOL CALLED: Setting price for {city} to ${price}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO prices (city, price) VALUES (?, ?) ON CONFLICT(city) DO UPDATE SET price = ?', (city.lower(), price, price))
        conn.commit()
    return f"Successfully set ticket price to {city} to ${price}"

# Test the functions
print("🧪 Testing tool functions:")
print(get_ticket_price("London"))  # Should show no data initially


🧪 Testing tool functions:
DATABASE TOOL CALLED: Getting price for London
No price data available for this city


In [5]:
# Tool definitions for OpenAI
get_price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

set_price_function = {
    "name": "set_ticket_price",
    "description": "Set the price of a return ticket to a destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city to set the price for",
            },
            "price": {
                "type": "number",
                "description": "The new price for the ticket",
            },
        },
        "required": ["destination_city", "price"],
        "additionalProperties": False
    }
}

# List of available tools
tools = [
    {"type": "function", "function": get_price_function},
    {"type": "function", "function": set_price_function}
]

print("🔧 Tools configured:")
print(f"   - {get_price_function['name']}: {get_price_function['description']}")
print(f"   - {set_price_function['name']}: {set_price_function['description']}")


🔧 Tools configured:
   - get_ticket_price: Get the price of a return ticket to the destination city.
   - set_ticket_price: Set the price of a return ticket to a destination city.


In [6]:
# Tool call handler
def handle_tool_calls(message):
    """Handle multiple tool calls from the LLM"""
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_ticket_price":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            price_details = get_ticket_price(city)
            responses.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })
        elif tool_call.function.name == "set_ticket_price":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            price = arguments.get('price')
            result = set_ticket_price(city, price)
            responses.append({
                "role": "tool",
                "content": result,
                "tool_call_id": tool_call.id
            })
    return responses

print("✅ Tool call handler configured!")


✅ Tool call handler configured!


In [7]:
# Main chat function
def chat(message, history):
    """Main chat function that handles tool calls"""
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    # Handle tool calls in a loop to support multiple consecutive tool calls
    while response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    return response.choices[0].message.content

print("✅ Chat function configured!")


✅ Chat function configured!


In [9]:
# Initialize sample data
def initialize_sample_data():
    """Initialize the database with sample ticket prices"""
    ticket_prices = {"london": 799, "paris": 899, "tokyo": 1420, "sydney": 2999, "new york": 1099, "los angeles": 1299, "san francisco": 1199, "chicago": 999, "houston": 1399, "miami": 1499, "washington": 1199, "boston": 1299, "philadelphia": 1099, "seattle": 1399, "san diego": 1299, "san jose": 1199, "austin": 1099, "san antonio": 1399, "san francisco": 1199, "san diego": 1299, "san jose": 1199, "austin": 1099, "san antonio": 1399, "nairobi": 1099, "cape town": 1299, "durban": 1199, "johannesburg": 1399, "pretoria": 1099, "bloemfontein": 1299, "polokwane": 1199, "port elizabeth": 1399, "port shepstone": 1099, "port saint john": 1299, "port elizabeth": 1199, "port shepstone": 1399, "port saint john": 1099}
    for city, price in ticket_prices.items():
        set_ticket_price(city, price)
    print("✅ Sample data initialized!")

# Initialize sample data
initialize_sample_data()

# Test the setup
print("\n🧪 Testing the setup:")
print(get_ticket_price("London"))
print(get_ticket_price("Tokyo"))


DATABASE TOOL CALLED: Setting price for london to $799
DATABASE TOOL CALLED: Setting price for paris to $899
DATABASE TOOL CALLED: Setting price for tokyo to $1420
DATABASE TOOL CALLED: Setting price for sydney to $2999
DATABASE TOOL CALLED: Setting price for new york to $1099
DATABASE TOOL CALLED: Setting price for los angeles to $1299
DATABASE TOOL CALLED: Setting price for san francisco to $1199
DATABASE TOOL CALLED: Setting price for chicago to $999
DATABASE TOOL CALLED: Setting price for houston to $1399
DATABASE TOOL CALLED: Setting price for miami to $1499
DATABASE TOOL CALLED: Setting price for washington to $1199
DATABASE TOOL CALLED: Setting price for boston to $1299
DATABASE TOOL CALLED: Setting price for philadelphia to $1099
DATABASE TOOL CALLED: Setting price for seattle to $1399
DATABASE TOOL CALLED: Setting price for san diego to $1299
DATABASE TOOL CALLED: Setting price for san jose to $1199
DATABASE TOOL CALLED: Setting price for austin to $1099
DATABASE TOOL CALLED: 

## Launch the Enhanced Airline Assistant

The assistant now supports both getting and setting ticket prices!


In [ ]:
# Launch the Gradio interface
print("🚀 Launching FlightAI Assistant with enhanced capabilities...")
print("📋 Available commands:")
print("   - 'What's the price to London?' (get price)")
print("   - 'Set the price to New York to $1200' (set price)")
print("   - 'Update Tokyo price to $1500' (set price)")
print("   - 'How much does it cost to go to Paris?' (get price)")

interface = gr.ChatInterface(
    fn=chat, 
    type="messages",
    title="FlightAI Assistant - Enhanced",
    description="Ask me about ticket prices or set new prices for destinations!",
    examples=[
        "What's the price to London?",
        "Set the price to New York to $1200",
        "How much does it cost to go to Paris?",
        "Update Tokyo price to $1500"
    ]
)

interface.launch()


🚀 Launching FlightAI Assistant with enhanced capabilities...
📋 Available commands:
   - 'What's the price to London?' (get price)
   - 'Set the price to New York to $1200' (set price)
   - 'Update Tokyo price to $1500' (set price)
   - 'How much does it cost to go to Paris?' (get price)
* Running on local URL:  http://127.0.0.1:7882
* To create a public link, set `share=True` in `launch()`.


DATABASE TOOL CALLED: Getting price for Paris
DATABASE TOOL CALLED: Setting price for Berlin to $9023


## Key Implementation Features

### 🔧 **Enhanced Tool Support**
- **Get Ticket Price**: Query current prices from database
- **Set Ticket Price**: Update prices in database
- **Multiple Tool Calls**: Handles both operations in sequence
- **Database Integration**: Persistent SQLite storage

### 🎯 **Tool Function Definitions**
```python
# Get Price Tool
get_price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

# Set Price Tool  
set_price_function = {
    "name": "set_ticket_price", 
    "description": "Set the price of a return ticket to a destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city to set the price for",
            },
            "price": {
                "type": "number", 
                "description": "The new price for the ticket",
            },
        },
        "required": ["destination_city", "price"],
        "additionalProperties": False
    }
}
```

### 🚀 **Usage Examples**
- **Get Price**: "What's the price to London?"
- **Set Price**: "Set the price to New York to $1200"
- **Update Price**: "Update Tokyo price to $1500"
- **Query Multiple**: "What are the prices to London and Paris?"

### 💾 **Database Schema**
```sql
CREATE TABLE prices (
    city TEXT PRIMARY KEY,
    price REAL
)
```

This implementation demonstrates advanced tool integration with OpenAI's function calling capabilities!
